# MA5701 Optimización no lineal: Tarea 2
**Fecha de entrega:** 4 de Julio a las 23:59 hrs.

**Profesor:** Jorge G. Amaya A. 

**Auxiliar:** Aldo Gutiérrez Concha. 

**Ayudantes:** Carolina Chiu y Mariano Vazquez.

**Autor:** Felipe Urrutia Vargas

In [2]:
!pip install numdifftools

In [3]:
import numpy as np
import numdifftools as nd

In [4]:
def f1(x):
    return 8 * ((x[0] - 6)**2) + (x[1] - 2)**4

In [7]:
grad = nd.Gradient(f1)
print(grad([0, 2]))

[-96.   0.]


In [52]:
A = np.array([
    [-1, 2],
    [3, 2],
    [-1, 0],
    [0, -1]
])
b = np.array([4, 12, 0, 0])

E = np.array([])
e = np.array([])

## I. Escriba un código computacional que haga operacional el método de direcciones admisibles (Zoutendijk)

Resolver el problema de optimización no lineal:
$$(P) \quad \begin{cases} \min &f (x) \\
s.a. &Ax \leq b \\
&Ex = e
\end{cases}$$

### Pseudo-algoritmo

(0) Sean $\epsilon>0$, $k = 0$, $x_0 \in \mathbb{R}^n$ tal que $Ax_0 \leq  b$, $Ex_0=e$ 

In [53]:
eps= 1e-2
x0 = [0, 2]
k = 0
xk = x0
# check factibilidad
feasible = np.all(A@xk <= b)
print(f"(0, k={k}) Factible: {feasible}")

(0, k=0) Factible: True


(1) Sea la descomposición $A = [A_1,A_2]^T, b =(b_1, b_2)^T$ tal que $A_1x_k = b_1, A_2x_k < b_2$.

In [54]:
# Step 1
index_A1 = np.isclose(A@xk, b)
index_A2 = ~index_A1

A1 = A[index_A1]
A2 = A[index_A2]

b1 = b[index_A1]
b2 = b[index_A2]

print(f"A1:\n{A1},\nb1:{b1}")
print(f"A2:\n{A2},\nb1:{b2}")

A1:
[[-1  2]
 [-1  0]],
b1:[4 0]
A2:
[[ 3  2]
 [ 0 -1]],
b1:[12  0]


(2) Resolver el problema lineal

$$(D_k) \quad \begin{cases} \min &\nabla f (x_k)^T d\\
s.a. &A_1d \leq 0 \\
&Ed = 0\\
&−1 \leq d_j \leq 1, \quad j = 1, . . . , n
\end{cases}$$
y sea $d_k$ solución de $(D_k)$.

Si $|| \nabla f (x_k)^T d_k || < \epsilon$, entonces parar.

Si no, ir a (3).

In [43]:
from scipy.optimize import linprog

In [72]:
# Step 2

obj = grad([0, 2])
print(f"∇𝑓(𝑥𝑘): {obj}")

lhs_ineq = A1

rhs_ineq = np.zeros(A1.shape[0])

lhs_eq = E if len(E) != 0 else None
rhs_eq = np.zeros(E.shape[0]) if len(E) != 0 else None

bnd = [(-1, 1) for _ in range(obj.shape[0])] 

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="simplex")
dk = opt.x
print(f"dk: {dk}")

∇𝑓(𝑥𝑘): [-96.   0.]
dk: [1.  0.5]
